In [ ]:
!git clone https://github.com/villano-lab/galactic-spin-W1.git

In [ ]:
import os
os.listdir()
os.chdir("galactic-spin-W1/binder")
os.listdir()
!!pip install lmfit dataPython


In [ ]:
# Importing libraries and dependencies
import sys
sys.path.append('python/')
# import load_galaxies as galdata
import components as comp                      # Functions for galaxy components.
import dataPython as dp

import numpy as np
import matplotlib.pyplot as plt                # Plotting
%matplotlib inline

import scipy.integrate as si                   # Integration
from ipywidgets import interactive, fixed, FloatSlider, HBox, Layout, Button, Label, Output, VBox
from IPython.display import display, clear_output
from IPython.display import Javascript

import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")              # Ignore warnings

In [ ]:
# Import measured velocity data
data = dp.getXYdata_wXYerr('data/NGC5533/noord-120kpc-datapoints.txt')
r_dat = np.asarray(data['xx'])              # radius
v_dat = np.asarray(data['yy'])              # velocity
v_err0 = np.asarray(data['ex'])             # error in radius
v_err1 = np.asarray(data['ey'])             # error in velocity

In [ ]:
# Plot 
plt.figure(figsize=(15,6))                                               # size of the galaxy
plt.errorbar(r_dat,v_dat,yerr=v_err1, marker='o', markersize=8, \
             ecolor='gray',color='gray', linestyle='none', linewidth=2)  # plot datapoints with errorbars
plt.xlabel('radius (kpc)',size=12)                                       # label x-axis
plt.ylabel('velocity (km/s)',size=12)                                    # label y-axis
plt.title(str('Observed Velocity of NGC 5533'), size=14)                 # title of the plot
plt.xlim(0,np.max(r_dat)+0.2)                                            # range of the x-axis
plt.ylim(0,np.max(v_dat)+100)                                            # range of the y-axis
plt.show()

In [ ]:
# Black hole component, traced curve
def blackhole(r,Mbh):
    return comp.blackhole(r,Mbh)

# Bulge component, traced curve
def bulge(r,bpref):
    return bpref*comp.bulge(r,1,'NGC5533',load=True)

# Disk component, traced curve
def disk(r,dpref):
    return comp.disk(r,dpref,'NGC5533')

# Gas component, traced curve
def gas(r,gpref):
    return comp.gas(r,gpref,'NGC5533')

# Dark matter component, traced curve
def halo(r,rc,rho0):
    return comp.halo(r,rc,rho0)

# Total velocity containing all components, added in quadrature
def total_all(r,Mbh,bpref,dpref,gpref,rc,rho0):
    total = np.sqrt(blackhole(r,Mbh)**2               # black hole
                    + bulge(r,bpref)**2               # bulge
                    + disk(r,dpref)**2                # disk
                    + gas(r,gpref)**2                 # gas
                    + halo(r,rc,rho0)**2)             # dark matter halo
    return total

# Total velocity of baryonic or luminous matter (no dark matter component), added in quadrature
def total_bary(r,Mbh,bpref,dpref,gpref):
    total = np.sqrt(blackhole(r,Mbh)**2               # black hole
                    + bulge(r,bpref)**2               # bulge
                    + disk(r,dpref)**2                # disk
                    + gas(r,gpref)**2)                # gas
    return total

# Scaling parameters

_,fit_dict = comp.bestfit(comp.totalvelocity_halo,'NGC5533')

best_Mbh = fit_dict['Mbh']
best_bpref = fit_dict['bpref']
best_dpref = fit_dict['dpref']
best_gpref = fit_dict['gpref']
best_rc = fit_dict['rcut']
best_rho0 = fit_dict['rho0']

# Equation for isothermal density
def density_iso(r,rc,rho0):
    density = rho0 * (1 + (r/rc)**2)**(-1)
    return density

# Equation for mass as a function of radius
def mass_function(r,rc,rho0):
    mass = 4 * np.pi * r**2 * density_iso(r,rc,rho0) 
    return mass

# Plot to visualize the mass distribution in the NGC 5533 galaxy for radii between 0 and 20 kpc
r = np.linspace(np.min(r_dat),np.max(r_dat),1000)
plt.figure(figsize=(10,6))                                                
plt.plot(r,mass_function(r,best_rc,best_rho0)) 
plt.xlabel('radius (kpc)',size=12)                                       
plt.ylabel('mass ($M_{sun}$)',size=12)                                   
plt.title(str('Mass of Dark Matter as a function of radius - NGC 5533'), size=14) 
plt.xlim(0,20)
plt.ylim(0,1e10)                                        
plt.show() 

In [ ]:
# Integrate to calculate total mass enclosed 
TotalMass = lambda rc,rho0: si.quad(mass_function, 0, np.max(r_dat), args=(rc,rho0))[0]

# Print total mass of Dark Matter in NGC 5533
print("The total mass of Dark Matter in the galaxy NGC 5533 is about {:.3e} solar masses.".format(TotalMass(best_rc,best_rho0)))

In [ ]:
# Plotting function
def f(rc,rho0):
        
    # Define radius
    r = np.linspace(np.min(r_dat),np.max(r_dat),1000)
    
    # Plot
    plt.figure(figsize=(11,7))
    plt.xlim(0,np.max(r_dat)+0.2)
    plt.ylim(0,np.max(v_dat)+100)
    
    plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')     # Measured data points
    plt.plot(r,blackhole(r,best_Mbh),label=("Central Supermassive Black Hole"),color='black') # Black hole component
    plt.plot(r,bulge(r,best_bpref),label=("Bulge"),color='orange')       # Bulge component
    plt.plot(r,disk(r,best_dpref),label=("Disk"),color='purple')         # Disk component 
    plt.plot(r,gas(r,best_gpref),label=("Gas"),color='blue')             # Gas component
    plt.plot(r,halo(r,rc,rho0),label=("Halo"),color='green')        # Dark matter halo component
    plt.plot(r,total_all(r,best_Mbh,best_bpref,best_dpref,best_gpref,rc,rho0),label=("Total Curve"),color='red')    # Total velocity with dark matter
    plt.plot(r,total_bary(r,best_Mbh,best_bpref,best_dpref,best_gpref),label=("Luminous Matter"),linestyle='--')    # Total velocity without dark matter
    plt.fill_between(r,galdata.NGC5533['n_band_btm'](r),galdata.NGC5533['n_band_top'](r),color='#dddddd',label="Confidence Band")    # Confidence band
    plt.title("Interactive Rotation Curve - Galaxy: NGC 5533",fontsize=20)
    plt.xlabel("Radius (kpc)")
    plt.ylabel("Velocity (km/s)")
    plt.legend(bbox_to_anchor=(1,1), loc="upper left")              # Put legend outside of the plot
    
    # Chi squared and reduced chi squared
    # Residuals
    residuals = v_dat - total_all(r_dat,best_Mbh,best_bpref,best_dpref,best_gpref,rc,rho0)
    # Error
    error = np.sqrt(v_err1**2 + galdata.NGC5533['n_v_bandwidth']**2)
    # Chi squared
    chisquared = np.sum(residuals**2/error**2)
    # Degrees of freedom
    dof = len(r_dat) - 6                 # number of degrees of freedom = number of observed data - number of fitting parameters
    # Reduced chi squared
    reducedchisquared = chisquared / dof 
    
    # Annotation
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(82,373,r"$\chi^2$: {:.5f}".format(chisquared)+'\n'+r"Reduced $\chi^2$: {:.5f}".format(reducedchisquared),bbox=props,size=10)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(69.5,350,r"Total Dark Matter Mass: {:.2e} $M_\odot$".format(TotalMass(rc,rho0)),bbox=props,size=10)
    plt.annotate('Data source: E. Noordermeer. The rotation curves of flattened Sérsic bulges. MNRAS,385(3):1359–1364, Apr 2008',
            xy=(0, 0), xytext=(0,5),
            xycoords=('axes fraction', 'figure fraction'),
            textcoords='offset points',
            size=10, ha='left', va='bottom')
    
    plt.show()

In [ ]:
# Appearance of widget
style = {'description_width': 'initial'}
layout = {'width':'600px'}

# Sliders for the widget
rc = FloatSlider(min=0.1, max=5, step=0.1, 
                    value=best_rc, 
                    description='Halo Core Radius [kpc]', 
                    readout_format='.2f', 
                    orientation='horizontal', 
                    style=style, layout=layout)
rho0 = FloatSlider(min=0, max=1e9, step=1e7, 
                    value=best_rho0, 
                    description=r'Halo Central Mass Density [$M_{\odot} / kpc^3$]', 
                    readout_format='.2e', 
                    orientation='horizontal', 
                    style=style, layout=layout)

# Interactive widget
def interactive_plot(f):
    interact = interactive(f,
                           rc = rc,
                           rho0 = rho0,
                           continuous_update=False)
    return interact

# Button to revert back to Best Fit
button = Button(
    description="Best Fit",
    button_style='warning',
    icon='check')
out = Output()

def on_button_clicked(_):
    rc.value = best_rc
    rho0.value = best_rho0
button.on_click(on_button_clicked)

In [ ]:
# Import dependencies
import sys
sys.path.append('python/')
import time
startTime = time.time()      # Calculate time for running this notebook
import numpy as np
import matplotlib.pyplot as plt 
import load_galaxies as lg   # Load load_galaxies.py library

In [ ]:
# NGC 5533
r_NGC5533, v_NGC5533, v_err_NGC5533 = lg.NGC5533['m_radii'],lg.NGC5533['m_velocities'],lg.NGC5533['m_v_errors']

# NGC 891
r_NGC0891, v_NGC0891, v_err_NGC0891 = lg.NGC0891['m_radii'],lg.NGC0891['m_velocities'],lg.NGC0891['m_v_errors']

# NGC 7814
r_NGC7814, v_NGC7814, v_err_NGC7814 = lg.NGC7814['m_radii'],lg.NGC7814['m_velocities'],lg.NGC7814['m_v_errors']

# NGC 5005
r_NGC5005, v_NGC5005, v_err_NGC5005 = lg.NGC5005['m_radii'],lg.NGC5005['m_velocities'],lg.NGC5005['m_v_errors']

# NGC 3198
r_NGC3198, v_NGC3198, v_err_NGC3198 = lg.NGC3198['m_radii'],lg.NGC3198['m_velocities'],lg.NGC3198['m_v_errors']

# UGC 477
r_UGC477, v_UGC477, v_err_UGC477 = lg.UGC477['m_radii'],lg.UGC477['m_velocities'],lg.UGC477['m_v_errors']

# UGC 1281
r_UGC1281, v_UGC1281, v_err_UGC1281 = lg.UGC1281['m_radii'],lg.UGC1281['m_velocities'],lg.UGC1281['m_v_errors']

# UGC 1437
r_UGC1437, v_UGC1437, v_err_UGC1437 = lg.UGC1437['m_radii'],lg.UGC1437['m_velocities'],lg.UGC1437['m_v_errors']

# UGC 2953
r_UGC2953, v_UGC2953, v_err_UGC2953 = lg.UGC2953['m_radii'],lg.UGC2953['m_velocities'],lg.UGC2953['m_v_errors']

# UGC 4325
r_UGC4325, v_UGC4325, v_err_UGC4325 = lg.UGC4325['m_radii'],lg.UGC4325['m_velocities'],lg.UGC4325['m_v_errors']

# UGC 5253
r_UGC5253, v_UGC5253, v_err_UGC5253 = lg.UGC5253['m_radii'],lg.UGC5253['m_velocities'],lg.UGC5253['m_v_errors']

# UGC 6787
r_UGC6787, v_UGC6787, v_err_UGC6787 = lg.UGC6787['m_radii'],lg.UGC6787['m_velocities'],lg.UGC6787['m_v_errors']

# UGC 10075
r_UGC10075, v_UGC10075, v_err_UGC10075 = lg.UGC10075['m_radii'],lg.UGC10075['m_velocities'],lg.UGC10075['m_v_errors']

In [ ]:
# Define radius for plotting
r = np.linspace(0,100,100)

# Plot
plt.figure(figsize=(14,8))                                        # size of the plot
plt.title('Measured radial velocity of multiple galaxies', fontsize=14)      # giving the plot a title
plt.xlabel('Radius (kpc)', fontsize=12)                           # labeling the x-axis
plt.ylabel('Velocity (km/s)', fontsize=12)                        # labeling the y-axis
plt.xlim(0,20)                                                    # limits of the x-axis (default from 0 to 20 kpc)
plt.ylim(0,420)                                                   # limits of the y-axis (default from 0 to 420 km/s)

# Plotting the measured data
plt.errorbar(r_NGC5533,v_NGC5533,yerr=v_err_NGC5533,    label='NGC 5533',  marker='o', markersize=6, linestyle='none', color='royalblue')
plt.errorbar(r_NGC0891,v_NGC0891,yerr=v_err_NGC0891,    label='NGC 891',   marker='o', markersize=6, linestyle='none', color='seagreen')
plt.errorbar(r_NGC7814,v_NGC7814,yerr=v_err_NGC7814,    label='NGC 7814',  marker='o', markersize=6, linestyle='none', color='m')
plt.errorbar(r_NGC5005,v_NGC5005,yerr=v_err_NGC5005,    label='NGC 5005',  marker='o', markersize=6, linestyle='none', color='red')
plt.errorbar(r_NGC3198,v_NGC3198,yerr=v_err_NGC3198,    label='NGC 3198',  marker='o', markersize=6, linestyle='none', color='gold')
plt.errorbar(r_UGC477,v_UGC477,yerr=v_err_UGC477,       label='UGC 477',   marker='o', markersize=6, linestyle='none', color='lightpink')
plt.errorbar(r_UGC1281,v_UGC1281,yerr=v_err_UGC1281,    label='UGC 1281',  marker='o', markersize=6, linestyle='none', color='aquamarine')
plt.errorbar(r_UGC1437,v_UGC1437,yerr=v_err_UGC1437,    label='UGC 1437',  marker='o', markersize=6, linestyle='none', color='peru')
plt.errorbar(r_UGC2953,v_UGC2953,yerr=v_err_UGC2953,    label='UGC 2953',  marker='o', markersize=6, linestyle='none', color='lightslategrey')
plt.errorbar(r_UGC4325,v_UGC4325,yerr=v_err_UGC4325,    label='UGC 4325',  marker='o', markersize=6, linestyle='none', color='darkorange')
plt.errorbar(r_UGC5253,v_UGC5253,yerr=v_err_UGC5253,    label='UGC 5253',  marker='o', markersize=6, linestyle='none', color='maroon')
plt.errorbar(r_UGC6787,v_UGC6787,yerr=v_err_UGC6787,    label='UGC 6787',  marker='o', markersize=6, linestyle='none', color='midnightblue')
plt.errorbar(r_UGC10075,v_UGC10075,yerr=v_err_UGC10075, label='UGC 10075', marker='o', markersize=6, linestyle='none', color='y')

plt.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.show()

In [ ]:
# Import dependencies
import sys
sys.path.append('python/')        # Define path
import time
startTime = time.time()           # Calculate time for running this notebook
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import ipywidgets as w
from matplotlib.widgets import RadioButtons
import load_galaxies as lg        # Load

galaxy = lg.NGC5533          # Options: lg.NGC5005 , lg.NGC5533 , lg.NGC7814 , lg.NGC0891 load_galaxies.py library

# Measured data
measured_data = galaxy['measured_data']

# Separate the columns of the data into arrays
m_radii = galaxy['m_radii']           # Radius
m_velocities = galaxy['m_velocities'] # Velocity
m_v_errors = galaxy['m_v_errors']     # Errors in velocity         

# Bulge spline
bulge_v = galaxy['bulge']['spline']   
# Disk spline
disk_v = galaxy['disk']['spline']
# Gas spline
gas_v = galaxy['gas']['spline']


# Set Parameters
massbh = galaxy['massbh']        # Mass of the central black hole in (solar mass) imported from our Python library
radius = 10                      # Radius to calculate velocity due to black hole component (in kpc)
G = 4.300e-6                     # Gravitational constant (kpc/solar mass*(km/s)^2)

# Display the mass of the black hole
if massbh != 0:
    print("Mass of the central supermassive black hole at the center of the galaxy {}: {:.3e} solar masses".format(galaxy['galaxyname'],massbh))

# Equation for point-mass rotation curve
def blackhole_v(r,massbh):
    return np.sqrt((G*massbh)/r)

# Displaying the value of the velocity of stars and gas at a chosen radius
if massbh == 0:
    print("Galaxy {} might have a central supermassive black hole but it is included in the bulge curve.".format(galaxy['galaxyname']))
if massbh != 0:
    print("Radial velocity at r={}[kpc]: {:.3f} km/s".format(radius,blackhole_v(radius,massbh)))


# Set parameters
rho0 = galaxy['rho0']   # central mass density (in solar mass/kpc^3)
rc = galaxy['rc']       # core radius (in kpc)
radius = 10             # radius (in kpc)

# Display the two dark matter parameters
print("Dark matter halo parameters: ")
print("     Central mass density: {:.2e} solar mass/kpc^3".format(rho0))
print("     Core radius: {:.2f} kpc".format(rc))

# Equation for dark matter halo velocity
def halo_v(r,rho0,rc):
    v = np.sqrt(4*np.pi*G*rho0*rc**2*(1 - rc/r * np.arctan(r/rc)))
    return v

# Display the velocity at the selected radius
print("     Velocity at {} kpc: {:.3f} km/s".format(radius,halo_v(radius,rho0,rc)))


# Scaling parameters
bpref = 0.75 #1      # Bulge 
dpref = 0.75 #1      # Disk
gpref = 1      # Gas
bhpref = 1     # Central supermassive black hole


# Total velocity WITHOUT dark matter halo component but with adjustable prefactors
def total_v_noDM(r,massbh):
    v = np.sqrt( (bpref * bulge_v(r))**2                   # bulge component + scaling parameter
               + (dpref * disk_v(r))**2                    # disk component + scaling parameter
               + (gpref * gas_v(r))**2                     # gas component + scaling parameter
               + (bhpref * blackhole_v(r,massbh))**2)      # central supermassive black hole component + scaling factor
    return v


# Total velocity WITH dark matter halo component
def total_v(r,massbh,rho0,rc):
    v = np.sqrt( 
        (bpref * bulge_v(r))**2                  # bulge component
        + (dpref * disk_v(r))**2                 # disk component
        + (gpref * gas_v(r))**2                  # gas component
        + (bhpref * blackhole_v(r,massbh))**2    # central black hole component (this might be zero for NGC 891, NGC 7814 and NGC 5005 but it is because it's included in the bulge velocity)
        + halo_v(r,rho0,rc)**2)                  # dark matter halo component
    return v



# Define radius for plotting
r_lowerlim = np.min(m_radii)                       # limit the radius to the min value of the measured radius
if r_lowerlim == 0:                                # getting rid of dividing by zero error
    r_lowerlim = 0.01
r_upperlim = np.max(m_radii)                       # limit the radius to the max value of the measured radius
r = np.linspace(r_lowerlim,r_upperlim,100)         # starting from zero results in a divide by zero error

# Plot 
plt.figure(figsize=(10.0,7.0))                     # size of the plot
plt.plot(r, bpref * bulge_v(r),            color='m', label='Bulge')
plt.plot(r, dpref * disk_v(r),             color='b', label='Stellar disk')                        
plt.plot(r, gpref * gas_v(r),              color='c', label='Gas')            
plt.plot(r, bhpref * blackhole_v(r,massbh), color='k', label='Central black hole')    # only for NGC 5533         
plt.plot(r, halo_v(r,rho0,rc),     color='g', label='Dark Matter Halo')   

plt.plot(r, total_v(r,massbh,rho0,rc), color='r',    label='Total curve with DM')
plt.plot(r, total_v_noDM(r,massbh),    color='gold', label='Total curve without DM')

plt.errorbar(m_radii, m_velocities, yerr=m_v_errors, 
             marker='o', markersize=8,
             linestyle='none', 
             label='{}'.format(galaxy['galaxyname']))

plt.title('Rotation curve of {}'.format(galaxy['galaxyname']), fontsize=16)     # giving the plot a title
plt.xlabel('Radius (kpc)', fontsize=12)                           # labeling the x-axis
plt.ylabel('Velocity (km/s)', fontsize=12)                        # labeling the y-axis
plt.xlim(0,r_upperlim)                                            # limits of the x-axis
plt.ylim(0,350)                                                   # limits of the y-axis
plt.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.show()